<a href="https://colab.research.google.com/github/ishika-thakur7802/PCOS-detection/blob/main/pcos_model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Add, Multiply, Activation, Reshape, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
# Data processing
img_data_list = []
labels = []

# Load data for class 0
data_path_0 = '/content/drive/MyDrive/PCOS-Hack/Dest_folder/0'
for dataset in os.listdir(data_path_0):
    input_img = cv2.imread(os.path.join(data_path_0, dataset))
    if input_img is None:
        print(f"Warning: Image {dataset} could not be loaded.")
        continue
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
    labels.append(0)  # Label for class 0
    input_img_resize = cv2.resize(input_img, (224, 224))
    input_img_resize = cv2.normalize(input_img_resize, None, 0, 255, cv2.NORM_MINMAX)
    img_data_list.append(input_img_resize)

# Load data for class 1
data_path_1 = '/content/drive/MyDrive/PCOS-Hack/Dest_folder/1_augmented'
for dataset in os.listdir(data_path_1):
    input_img = cv2.imread(os.path.join(data_path_1, dataset))
    if input_img is None:
        print(f"Warning: Image {dataset} could not be loaded.")
        continue
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
    labels.append(1)  # Label for class 1
    input_img_resize = cv2.resize(input_img, (224, 224))
    input_img_resize = cv2.normalize(input_img_resize, None, 0, 255, cv2.NORM_MINMAX)
    img_data_list.append(input_img_resize)


In [ ]:
# img_data_list = []
# labels = []

# input_img = cv2.imread('/content/f1.jpeg')
# input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
# labels.append(1)  # Label for class 1
# input_img_resize = cv2.resize(input_img, (224, 224))

# input_img_resize = cv2.normalize(input_img_resize, None, 0, 255, cv2.NORM_MINMAX)
# img_data_list.append(input_img_resize)

In [ ]:
print(f'Number of images: {len(img_data_list)}')
print(f'Number of labels: {len(labels)}')


Number of images: 5006
Number of labels: 5006


In [ ]:
# Convert lists to arrays
img_data = np.array(img_data_list)
labels = np.array(labels)

print(f'Image data shape: {img_data.shape}')
print(f'Labels shape: {labels.shape}')


Image data shape: (5006, 224, 224, 3)
Labels shape: (5006,)


In [ ]:
# One-hot encode the labels
labels = to_categorical(labels, num_classes=2)

In [ ]:
print(f'Labels shape after one-hot encoding: {labels.shape}')

Labels shape after one-hot encoding: (5006, 2)


In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(img_data, labels, test_size=0.2, random_state=42)

In [ ]:
print(f'X_train shape after split: {X_train.shape}')
print(f'y_train shape after split: {y_train.shape}')
print(f'X_val shape after split: {X_val.shape}')
print(f'y_val shape after split: {y_val.shape}')

X_train shape after split: (4004, 224, 224, 3)
y_train shape after split: (4004, 2)
X_val shape after split: (1002, 224, 224, 3)
y_val shape after split: (1002, 2)


In [ ]:
assert X_train.shape[0] == y_train.shape[0], "Mismatch in number of samples between X_train and y_train"
assert X_val.shape[0] == y_val.shape[0], "Mismatch in number of samples between X_val and y_val"

In [ ]:
# X_train = img_data[0]
# y_train = labels[0]

In [ ]:
def cam_block(x):
    print("Input shape:", x.shape)

    # Apply convolution with dilation rates
    dilated_1 = Conv2D(x.shape[-1], (3, 3), activation='relu', dilation_rate=1, padding='same')(x)
    print("dilated_1 shape:", dilated_1.shape)

    dilated_2 = Conv2D(x.shape[-1], (3, 3), activation='relu', dilation_rate=2, padding='same')(x)
    print("dilated_2 shape:", dilated_2.shape)

    dilated_3 = Conv2D(x.shape[-1], (3, 3), activation='relu', dilation_rate=3, padding='same')(x)
    print("dilated_3 shape:", dilated_3.shape)

    # Element-wise add conv_1 and conv_2
    add_result = Add()([dilated_1, dilated_2])
    print("add_result shape:", add_result.shape)

    # Element-wise multiply conv_2 and conv_3
    mul_result = Multiply()([dilated_2, dilated_3])
    print("mul_result shape:", mul_result.shape)

    # 1x1 convolutions on the results
    conv_add = Conv2D(x.shape[-1], (1, 1), activation='relu', padding='same')(add_result)
    print("conv_add shape:", conv_add.shape)
    conv_mul = Conv2D(x.shape[-1], (1, 1), activation='relu', padding='same')(mul_result)
    print("conv_mul shape:", conv_mul.shape)

    # Apply Global Average Pooling and Global Max Pooling
    gap = GlobalAveragePooling2D()(conv_add)
    print("gap shape:", gap.shape)
    gmp = GlobalMaxPooling2D()(conv_mul)
    print("gmp shape:", gmp.shape)

    # Multiply the pooled outputs
    combined = Multiply()([gap, gmp])
    print("combined shape:", combined.shape)

    # Pass through sigmoid activation function
    sigmoid_output = Activation('sigmoid')(combined)
    print("sigmoid_output shape:", sigmoid_output.shape)

    # Multiply with the initial input
    multiplied_output = Multiply()([x, sigmoid_output])
    print("multiplied_output shape:", multiplied_output.shape)

    # Element-wise add the multiplied output to the initial input
    final_output = Add()([x, multiplied_output])
    print("final_output shape:", final_output.shape)

    return final_output

In [ ]:
from tensorflow.keras.layers import Concatenate

# Define the main model with CAM blocks
input_shape = (224, 224, 3)
num_classes = 2

inputs = Input(shape=input_shape)

# First convolutional layer with dilation rate 1
x1 = Conv2D(32, (3, 3), activation='relu', dilation_rate=1, padding='same')(inputs)

# Second convolutional layer with dilation rate 2
x2 = Conv2D(32, (3, 3), activation='relu', dilation_rate=2, padding='same')(x1)

# Third convolutional layer with dilation rate 3
x3 = Conv2D(64, (3, 3), activation='relu', dilation_rate=3, padding='same')(x2)

# Fourth convolutional layer with dilation rate 4
x4 = Conv2D(64, (3, 3), activation='relu', dilation_rate=4, padding='same')(x3)

# Fifth convolutional layer with dilation rate 5
x5 = Conv2D(64, (3, 3), activation='relu', dilation_rate=5, padding='same')(x4)

# Apply the CAM block to each convolutional output
cam_output1 = cam_block(x1)
cam_output2 = cam_block(x2)
cam_output3 = cam_block(x3)
cam_output4 = cam_block(x4)
cam_output5 = cam_block(x5)

# Apply Global Max Pooling to each CAM output
gmp1 = GlobalMaxPooling2D()(cam_output1)
gmp2 = GlobalMaxPooling2D()(cam_output2)
gmp3 = GlobalMaxPooling2D()(cam_output3)
gmp4 = GlobalMaxPooling2D()(cam_output4)
gmp5 = GlobalMaxPooling2D()(cam_output5)

# Feature aggregation layer
aggregated_features = Concatenate()([gmp1, gmp2, gmp3, gmp4, gmp5])

# Add a Dense layer with 32 units
dense_features = Dense(32, activation='relu')(aggregated_features)

# Add another Dense layer with 2 units for classification
dense_output = Dense(num_classes, activation='softmax')(dense_features)

# Create the model
model = Model(inputs, dense_output)


Input shape: (None, 224, 224, 32)
dilated_1 shape: (None, 224, 224, 32)
dilated_2 shape: (None, 224, 224, 32)
dilated_3 shape: (None, 224, 224, 32)
add_result shape: (None, 224, 224, 32)
mul_result shape: (None, 224, 224, 32)
conv_add shape: (None, 224, 224, 32)
conv_mul shape: (None, 224, 224, 32)
gap shape: (None, 32)
gmp shape: (None, 32)
combined shape: (None, 32)
sigmoid_output shape: (None, 32)
multiplied_output shape: (None, 224, 224, 32)
final_output shape: (None, 224, 224, 32)
Input shape: (None, 224, 224, 32)
dilated_1 shape: (None, 224, 224, 32)
dilated_2 shape: (None, 224, 224, 32)
dilated_3 shape: (None, 224, 224, 32)
add_result shape: (None, 224, 224, 32)
mul_result shape: (None, 224, 224, 32)
conv_add shape: (None, 224, 224, 32)
conv_mul shape: (None, 224, 224, 32)
gap shape: (None, 32)
gmp shape: (None, 32)
combined shape: (None, 32)
sigmoid_output shape: (None, 32)
multiplied_output shape: (None, 224, 224, 32)
final_output shape: (None, 224, 224, 32)
Input shape: (None

In [ ]:
from tensorflow.keras import backend as K

# Clear the session to free up memory
K.clear_session()


In [ ]:
# Check the shapes of the training and validation data
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'y_val shape: {y_val.shape}')

# The shapes should match in the first dimension (number of samples)
# Ensure all arrays contain the same number of samples
assert X_train.shape[0] == y_train.shape[0], "Mismatch in number of samples between X_train and y_train"
assert X_val.shape[0] == y_val.shape[0], "Mismatch in number of samples between X_val and y_val"

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=16,
                    validation_data=(X_val, y_val))

# Plot the training and validation accuracy and loss
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss')

plt.show()


X_train shape: (4004, 224, 224, 3)
y_train shape: (4004, 2)
X_val shape: (1002, 224, 224, 3)
y_val shape: (1002, 2)
Epoch 1/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 329s 1s/step - accuracy: 0.6735 - loss: 7.0671 - val_accuracy: 0.7445 - val_loss: 0.7335
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 237s 937ms/step - accuracy: 0.7754 - loss: 0.6568 - val_accuracy: 0.7375 - val_loss: 0.5553
Epoch 3/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 235s 937ms/step - accuracy: 0.8082 - loss: 0.4650 - val_accuracy: 0.8703 - val_loss: 0.3654
Epoch 4/50
 90/251 ━━━━━━━━━━━━━━━━━━━━ 2:22 882ms/step - accuracy: 0.8284 - loss: 0.3750

In [ ]:
model.save('/content/drive/MyDrive/PCOS-Hack/models/model_50.h5')
